## 환경 초기 설정

**중요: 아래 셀에 현재 실습에서 사용하는 Google Cloud 프로젝트 ID를 반드시 입력해야 합니다. 이 값은 실습 환경 내의 모든 리소스에 액세스하기 위한 필수 값입니다. 이 노트북에서 첫 번째로 실행해야 하는 셀입니다.**



In [ ]:
# 이 셀에 프로젝트 아이디를 입력합니다.
PROJECT_ID = 'your-gcp-project-id' # <-- 여기에 프로젝트 ID를 입력해주세요!

# PROJECT_ID가 공란이 아니도록 확인합니다. 공란일 경우 에러가 발생합니다.
if not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please enter your Project ID above.")

print(f"Project ID set to: {PROJECT_ID}")

이제 이 셀을 실행하여 태스크 2 환경을 초기화합니다. 이 셀은 태스크 2에 필요한 라이브러리를 가져오고, BigQuery 클라이언트를 초기화하며, 사용할 주요 변수(테이블 ID 등)를 정의합니다.

In [ ]:
# 태스크 2에 필요한 라이브러리를 가져오고 클라이언트 및 변수를 초기화합니다.
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
from IPython.display import display

# 위의 셀에 PROJECT_ID가 정의되어 있는지 확인
# 이 셀을 실행하기 전, 위에서 프로젝트 ID를 입력하는 셀을 반드시 실행해야 합니다.
if 'PROJECT_ID' not in locals() or not PROJECT_ID:
    raise ValueError("ERROR: PROJECT_ID is not set. Please run the 'Set Your Project ID' cell above first.")

client = bigquery.Client(project=PROJECT_ID, location="us-central1")

DATASET_ID = 'cymbal'

TABLE_ID_CUSTOMERS = f"{PROJECT_ID}.{DATASET_ID}.customers"
table_id_multimodal_reviews = f"{PROJECT_ID}.{DATASET_ID}.multimodal_customer_reviews"
GEMINI_MODEL_NAME = f'{PROJECT_ID}.{DATASET_ID}.gemini_flash_model'
table_id_segment_level_analysis = f"{PROJECT_ID}.{DATASET_ID}.segment_level_gemini_analysis"

print(f"BigQuery Client Initialized for Project ID: {PROJECT_ID}")

def run_bq_query(sql: str, client: bigquery.Client):
    try:
        query_job = client.query(sql)
        print(f"Job {query_job.job_id} in state {query_job.state}")
        if query_job.statement_type == 'SELECT':
            df = query_job.to_dataframe()
            print(f"Query complete. Fetched {len(df)} rows.")
            return df
        else:
            query_job.result()
            print(f"Query for statement type {query_job.statement_type} complete.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## 고객 데모그래픽 테이블 확인

In [ ]:
%%bigquery
SELECT customer_id, first_name, age, gender, loyalty_member FROM `cymbal.customers` LIMIT 5

## 페르소나 프로필 식별
### 이 쿼리는 페르소나 프로필로 사용할 고객 속성의 고유한 조합을 식별합니다.

In [ ]:
sql_get_profiles = f"""
WITH EnrichedData AS (
    SELECT
        c.customer_id,
        JSON_EXTRACT_SCALAR(mcr.sentiment_json_string, '$.sentiment') as text_sentiment,
        CASE
            WHEN c.age < 40 THEN 'Younger_Adult'
            ELSE 'Older_Adult'
        END AS age_group,
        UPPER(c.gender) as gender,
        IF(c.loyalty_member, 'LOYAL', 'NON_LOYAL') as loyalty_status
    FROM `{table_id_multimodal_reviews}` AS mcr
    JOIN `{TABLE_ID_CUSTOMERS}` AS c ON mcr.customer_id = c.customer_id
    WHERE c.age IS NOT NULL AND c.gender IS NOT NULL AND c.loyalty_member IS NOT NULL
)
SELECT DISTINCT
    CONCAT(age_group, '_', gender, '_', loyalty_status) as persona_age_group_profile
FROM EnrichedData
ORDER BY 1;
"""
print(f"Identifying unique segment profiles for Gemini analysis...")
df_profiles = run_bq_query(sql_get_profiles, client)
if df_profiles is not None:
    display(df_profiles)

## 페르소나 데이터 시각화를 통한 EDA
### 이 쿼리는 각 세그먼트의 고객 수를 집계하고 해당 데이터를 막대 차트로 시각화합니다.

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_customer_distribution(df: pd.DataFrame):
    if df is None or df.empty:
        print("DataFrame is empty. Skipping plot generation.")
        return

    sns.set_theme(style="whitegrid", font_scale=1.0)

    plt.figure(figsize=(10, 6))

    palette = sns.cubehelix_palette(n_colors=len(df), start=.5, rot=-.75, dark=0.3, light=0.7)

    ax = sns.barplot(
        x='persona_age_group_profile', y='customer_count', data=df,
        palette=palette, hue='persona_age_group_profile', legend=False
    )

    for p in ax.patches:
        ax.annotate(f'{int(p.get_height()):,}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 8), textcoords='offset points',
                    fontsize=9, color='dimgray')

    ax.set(title='Customer Segment Distribution', xlabel='Persona Profile', ylabel='Number of Customers')
    ax.title.set_size(16); ax.title.set_weight('bold'); ax.xaxis.label.set_size(12);
    ax.yaxis.label.set_size(12); ax.title.set_position([.5, 1.05]);

    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, df['customer_count'].max() * 1.15)
    sns.despine()
    plt.tight_layout()
    plt.show()

In [ ]:
sql_data_for_viz = f"""
WITH EnrichedData AS (
    SELECT
        c.customer_id,
        CASE
            WHEN c.age < 40 THEN 'Younger_Adult'
            ELSE 'Older_Adult'
        END AS age_group,
        UPPER(c.gender) as gender,
        IF(c.loyalty_member, 'LOYAL', 'NON_LOYAL') as loyalty_status
    FROM `{TABLE_ID_CUSTOMERS}` AS c
    WHERE c.age IS NOT NULL AND c.gender IS NOT NULL AND c.loyalty_member IS NOT NULL
)
SELECT
    CONCAT(age_group, '_', gender, '_', loyalty_status) as persona_age_group_profile,
    COUNT(DISTINCT customer_id) AS customer_count
FROM EnrichedData
GROUP BY persona_age_group_profile
ORDER BY customer_count DESC;
"""

print("Querying data for visualization...")
df_for_viz = run_bq_query(sql_data_for_viz, client)

print("Generating plot...")
plot_customer_distribution(df_for_viz)

## Gemini를 사용하여 상세 페르소나 생성
### 해당 프롬프트는 한 번의 호출만으로 온전한 페르소나 분석을 생성하도록 설계되었습니다.

In [ ]:
# The Gemini prompt is designed to generate a complete persona analysis in a single call.
# The prompt string is also properly escaped for SQL to prevent syntax errors.
gemini_prompt_template = """
고객 세그먼트 프로필 "{p}"를 기반으로 유효한 단일 JSON 객체를 생성하세요.
JSON은 다음 키를 반드시 포함해야 합니다:
"persona_description" (이 페르소나에 대한 간결한 한 문장 요약),
"summary" (그들의 예상되는 선호도에 대한 더 자세한 요약),
"motivations" (구매 결정에 영향을 미치는 요인),
"needs" (제품 또는 서비스에서 찾는 것),
"marketing_pitch" (그들을 타겟팅하는 짧은 마케팅 문구).
전체 출력은 오직 이 단일 JSON 객체여야 합니다.
"""


all_results = []
# 옵션에 따라 df_profiles 또는 cymbal.unique_segment_profiles 테이블을 사용합니다.
# BigQuery Studio 옵션의 경우 df_profiles가 이미 메모리에 로드되어 있습니다.
# Data Canvas 옵션의 경우 cymbal.unique_segment_profiles 테이블에서 데이터를 로드해야 합니다.
if 'df_profiles' in locals() and df_profiles is not None:
    source_df = df_profiles
    print("Using df_profiles (from BigQuery Studio option) for Gemini analysis.")
else:
    # Data Canvas 옵션을 선택한 경우, 여기서 테이블에서 데이터를 로드합니다.
    print("Loading persona profiles from cymbal.unique_segment_profiles (from Data Canvas option) for Gemini analysis.")
    sql_load_profiles = f"SELECT persona_age_group_profile FROM `cymbal.unique_segment_profiles` ORDER BY 1"
    source_df = run_bq_query(sql_load_profiles, client)


if source_df is not None:
    print(f"Starting Gemini analysis for {len(source_df)} profiles...")
    for _, row in source_df.iterrows():
        profile = row['persona_age_group_profile']


        # Format the prompt with the profile name
        raw_prompt = gemini_prompt_template.format(p=profile)


        # Make the prompt string safe for embedding in a SQL query by escaping quotes and newlines.
        prompt_for_sql = raw_prompt.replace("'", "''").replace("\n", " ")


        sql = f"""
        SELECT
            '{profile}' AS profile,
            ml_generate_text_llm_result AS analysis
        FROM
            ML.GENERATE_TEXT(
                MODEL `{GEMINI_MODEL_NAME}`,
                (SELECT '{prompt_for_sql}' AS prompt),
                STRUCT(0.5 AS temperature, 1024 AS max_output_tokens, TRUE AS flatten_json_output)
            )
        """


        print(f"  Analyzing profile: '{profile}'...")
        result_df = run_bq_query(sql, client)
        if result_df is not None:
            all_results.append(result_df)


    if all_results:
        df_all_analysis = pd.concat(all_results, ignore_index=True)
        print(f"\nSaving {len(df_all_analysis)} analyses to 'cymbal.segment_level_gemini_analysis'...")
        # Use the recommended pandas_gbq library to write to BigQuery
        pandas_gbq.to_gbq(df_all_analysis, f'{DATASET_ID}.segment_level_gemini_analysis', project_id=PROJECT_ID, if_exists='replace')
        print("Successfully saved results.")


## 생성된 페르소나 분석 테이블 확인
### Gemini로 분석한 원본 테이블의 내용을 표시하여 확인합니다.

In [ ]:
# This allows us to inspect the 'analysis' column before parsing it.
print(f"\n--- Verifying Raw Gemini Output in: {table_id_segment_level_analysis} ---")
df_raw_analysis = run_bq_query(f"SELECT * FROM `{table_id_segment_level_analysis}` LIMIT 5", client)
if df_raw_analysis is not None:
    with pd.option_context('display.max_colwidth', None):
        display(df_raw_analysis)

## 최종 인사이트 및 페르소나 테이블 생성

In [ ]:
# This query combines all data sources into a single, comprehensive table for final analysis.
table_id_final_customer_insights = f"{PROJECT_ID}.{DATASET_ID}.final_customer_insights"
sql_create_final_table = f"""
CREATE OR REPLACE TABLE `{table_id_final_customer_insights}` AS
WITH EnrichedData AS (
    SELECT mcr.*, c.first_name, c.last_name, c.age, c.gender, c.loyalty_member,
        CONCAT(
            CASE WHEN c.age < 40 THEN 'Younger_Adult' ELSE 'Older_Adult' END, '_',
            UPPER(c.gender), IF(c.loyalty_member, '_LOYAL', '_NON_LOYAL')
        ) AS persona_age_group_profile
    FROM `{table_id_multimodal_reviews}` AS mcr
    JOIN `{TABLE_ID_CUSTOMERS}` AS c ON mcr.customer_id = c.customer_id
)
SELECT enriched.*, persona.analysis AS gemini_persona_analysis
FROM EnrichedData enriched
LEFT JOIN `{table_id_segment_level_analysis}` persona ON enriched.persona_age_group_profile = persona.profile;
"""
print(f"Creating the final customer insights table '{table_id_final_customer_insights}'...")
run_bq_query(sql_create_final_table, client)


# This query now creates the persona definitions table by robustly parsing the description
# from the JSON generated by Gemini, cleaning it first to handle any markdown.
final_persona_table_id = f"{PROJECT_ID}.{DATASET_ID}.customer_persona_definitions"
sql_create_personas = f"""
CREATE OR REPLACE TABLE `{final_persona_table_id}` AS
WITH cleaned_analysis AS (
  SELECT
    profile,
    -- Clean the JSON string by removing markdown backticks and whitespace
    TRIM(REGEXP_REPLACE(analysis, r'(?i)(^```json\\s*|\\s*```$)', '')) as cleaned_json
  FROM
    `{table_id_segment_level_analysis}`
)
SELECT
    profile AS persona_age_group_profile,
    JSON_EXTRACT_SCALAR(cleaned_json, '$.persona_description') AS persona_segment_description
FROM
    cleaned_analysis
WHERE
    JSON_EXTRACT_SCALAR(cleaned_json, '$.persona_description') IS NOT NULL;
"""
print(f"\nCreating final persona definitions table from Gemini output: {final_persona_table_id}...")
run_bq_query(sql_create_personas, client)

print(f"\n--- Final Customer Persona Definitions (Generated by Gemini) ---")
df_personas = run_bq_query(f"SELECT * FROM `{final_persona_table_id}` ORDER BY 1", client)
if df_personas is not None:
    with pd.option_context('display.max_colwidth', None):
        display(df_personas)